pip install python-binance
pip install ta
pip install pandas_ta
pip install termcolor

In [61]:
import pandas as pd
from binance.client import Client
import ta
import pandas_ta as pda
import matplotlib.pyplot as plt
import numpy as np
from termcolor import colored


# ----- Functions Definition -------
def get_chop(high, low, close, window):
    tr1 = pd.DataFrame(high - low).rename(columns = {0:'tr1'})
    tr2 = pd.DataFrame(abs(high - close.shift(1))).rename(columns = {0:'tr2'})
    tr3 = pd.DataFrame(abs(low - close.shift(1))).rename(columns = {0:'tr3'})
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').dropna().max(axis = 1)
    atr = tr.rolling(1).mean()
    highh = high.rolling(window).max()
    lowl = low.rolling(window).min()
    ci = 100 * np.log10((atr.rolling(window).sum()) / (highh - lowl)) / np.log10(window)
    return ci

client = Client()

klinesT = client.get_historical_klines("ETHUSDT", Client.KLINE_INTERVAL_1HOUR, "01 january 2020")

df = pd.DataFrame(klinesT, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
df['close'] = pd.to_numeric(df['close'])
df['high'] = pd.to_numeric(df['high'])
df['low'] = pd.to_numeric(df['low'])
df['open'] = pd.to_numeric(df['open'])

df = df.set_index(df['timestamp'])
df.index = pd.to_datetime(df.index, unit='ms')
del df['timestamp']
df



,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
timestamp,,,,,,,,,,,
2020-01-01 00:00:00,129.16,129.19,128.68,128.87,7769.17336000,1577840399999,1000929.74221140,2504,4149.93345000,534619.33896550,0
2020-01-01 01:00:00,128.87,130.65,128.78,130.64,11344.65516000,1577843999999,1474278.48163660,4885,5930.54276000,770486.05667730,0
2020-01-01 02:00:00,130.63,130.98,130.35,130.85,7603.35623000,1577847599999,994025.61405700,3046,3324.35218000,434675.44465540,0
2020-01-01 03:00:00,130.85,130.89,129.94,130.20,4968.55433000,1577851199999,647360.95286070,2818,1810.03564000,235890.33019660,0
2020-01-01 04:00:00,130.21,130.74,130.15,130.20,3397.90747000,1577854799999,443006.65068500,2264,1839.74371000,239848.34833520,0
...,...,...,...,...,...,...,...,...,...,...,...
2021-10-25 07:00:00,4124.12,4177.62,4113.11,4149.33,21599.02970000,1635148799999,89717772.83668200,57937,10357.24930000,43026624.33764000,0
2021-10-25 08:00:00,4149.33,4162.12,4120.25,4123.60,17859.66910000,1635152399999,74024200.96228000,48592,9083.09630000,37655740.47105100,0
2021-10-25 09:00:00,4123.60,4145.29,4120.51,4134.85,9240.37710000,1635155999999,38190818.14491700,26448,4499.18030000,18598161.78174100,0


# Backtest Execution

In [62]:
df.drop(df.columns.difference(['open','high','low','close','volume']), 1, inplace=True)

#Simple Moving Average
# df['SMA']=ta.trend.sma_indicator(df['close'], window=12)

#Exponential Moving Average
df['SMA200']=ta.trend.sma_indicator(df['close'],50)
df['SMA600']=ta.trend.sma_indicator(df['close'], 150)

# #Relative Strength Index (RSI)
#df['RSI'] =ta.momentum.rsi(close=df['close'], window=14)

# #MACD
# MACD = ta.trend.MACD(close=df['close'], window_fast=12, window_slow=26, window_sign=9)
# df['MACD'] = MACD.macd()
# df['MACD_SIGNAL'] = MACD.macd_signal()
# df['MACD_DIFF'] = MACD.macd_diff() #Histogramme MACD

# #Stochastic RSI
# df['STOCH_RSI'] = ta.momentum.stochrsi(close=df['close'], window=14, smooth1=3, smooth2=3) #Non moyenné 
# df['STOCH_RSI_D'] = ta.momentum.stochrsi_d(close=df['close'], window=14, smooth1=3, smooth2=3) #Orange sur TradingView
# df['STOCH_RSI_K'] =ta.momentum.stochrsi_k(close=df['close'], window=14, smooth1=3, smooth2=3) #Bleu sur TradingView

# #Ichimoku
# df['KIJUN'] = ta.trend.ichimoku_base_line(high=df['high'], low=df['low'], window1=9, window2=26)
# df['TENKAN'] = ta.trend.ichimoku_conversion_line(high=df['high'], low=df['low'], window1=9, window2=26)
# df['SSA'] = ta.trend.ichimoku_a(high=df['high'], low=df['low'], window1=9, window2=26)
# df['SSB'] = ta.trend.ichimoku_b(high=df['high'], low=df['low'], window2=26, window3=52)

# #Bollinger Bands
# BOL_BAND = ta.volatility.BollingerBands(close=df['close'], window=20, window_dev=2)
# df['BOL_H_BAND'] = BOL_BAND.bollinger_hband() #Bande Supérieur
# df['BOL_L_BAND'] = BOL_BAND.bollinger_lband() #Bande inférieur
# df['BOL_MAVG_BAND'] = BOL_BAND.bollinger_mavg() #Bande moyenne

# #Average True Range (ATR)
# df['ATR'] = ta.volatility.average_true_range(high=df['high'], low=df['low'], close=df['close'], window=14)

# #Super Trend
# ST_length = 10
# ST_multiplier = 3.0
# superTrend = pda.supertrend(high=df['high'], low=df['low'], close=df['close'], length=ST_length, multiplier=ST_multiplier)
# df['SUPER_TREND'] = superTrend['SUPERT_'+str(ST_length)+"_"+str(ST_multiplier)] #Valeur de la super trend
# df['SUPER_TREND_DIRECTION'] = superTrend['SUPERTd_'+str(ST_length)+"_"+str(ST_multiplier)] #Retourne 1 si vert et -1 si rouge

# #Awesome Oscillator
# df['AWESOME_OSCILLATOR'] = ta.momentum.awesome_oscillator(high=df['high'], low=df['low'], window1=5, window2=34)

# # Kaufman’s Adaptive Moving Average (KAMA)
# df['KAMA'] = ta.momentum.kama(close=df['close'], window=10, pow1=2, pow2=30)

# #Choppiness index
# df['CHOP'] = get_chop(high=df['high'], low=df['low'], close=df['close'], window=14)  

df=df.iloc[700:,:]
df.isna().sum()

open      0
high      0
low       0
close     0
volume    0
SMA200    0
SMA600    0
dtype: int64

# Spot Backtest

4128.98

In [68]:
dfTest = df.copy()
frais=0.007
btc=0.2
usdt=1000
lastindex=df.first_valid_index()

for index,row in df.iterrows():
    if df['SMA200'][lastindex] > df['SMA600'][lastindex] and usdt>10:
        btc =usdt/df['close'][index]
        btc = btc - (frais*btc)
        usdt=0
        print(btc)
        
    if df['SMA200'][lastindex] > df['SMA600'][lastindex] and btc > 0.0001:
        usdt =btc*df['close'][index]
        usdt = usdt -(0.007*usdt)
        btc=0
    lastindex=index

usdt +btc*df['close'].iloc[-1]

5.698708751793401
5.632134926660916
5.548454583002087
5.4064232987840075
5.3346336785711825
5.264398507758909
5.2218410508889
5.13488995536686
5.030533256722583
4.94158429520664
4.910077074312082
4.7897891909782535
4.720309458954619
4.606507071096175
4.504366684302382
4.4314930648631075
4.293265733115407
4.223539159797346
4.131149700908144
4.116966280940323
4.072982861557534
4.018125210842472
3.9791536844177156
3.9189343674753307
3.8882155369735862
3.855757372003353
3.7612940887802555
3.7238604273121125
3.6664838688784647
3.626047201061625
3.587075245343725
3.5511096977346246
3.5584162760023736
3.5044417213908394
3.461377508948421
3.373087746206177
3.35510087977676
3.3092209239146246
3.2441451058779323
3.199955839424621
3.1362590092567055
3.0848143868320035
3.026224734286843
3.0153396754523065
2.9552120391435
2.8716442103950683
2.8297340130217092
2.7958822172800137
2.780513731235457
2.7483690186761716
2.701691185249418
2.648580026482691
2.6395923864166373
2.60061448765514
2.56461616661

9.970272148420568

In [66]:
dfTest = df.copy()
# dfTest = df['2021-01-01':]
dt = None
dt = pd.DataFrame(columns = ['date','position', 'reason', 'price', 'frais' ,'fiat', 'coins', 'wallet', 'drawBack'])

usdt = 1000
initalWallet = usdt
coin = 0
wallet = 1000
lastAth = 0
previousRow = dfTest.iloc[0]
makerFee = 0.0005
takerFee = 0.0007
stopLoss = 0
takeProfit = 500000
buyReady = True
sellReady = True


def buyCondition(row, previousRow):
    if row['SMA200'] > row['SMA600']:
        return True
    else:
        return False
    
def sellCondition(row, previousRow):
    if row['SMA600'] > row['SMA200']:
        return True
    else:
        return False

for index, row in dfTest.iterrows():
  #Buy market order
    if buyCondition(row, previousRow) and usdt > 0 and buyReady == True:
        #You can define here at what price you buy
        buyPrice = row['close']

        #Define the price of you SL and TP or comment it if you don't want a SL or TP
        stopLoss = buyPrice - 0.02 * buyPrice
        takeProfit = buyPrice + 0.1 * buyPrice

        coin = usdt / buyPrice
        fee = takerFee * coin
        coin = coin - fee
        usdt = 0
        wallet = coin * row['close']
        if wallet > lastAth:
            lastAth = wallet

        # print("Buy COIN at",buyPrice,'$ the', index)
        myrow = {'date': index,'position': "Buy", 'reason': 'Buy Market','price': buyPrice,'frais': fee*row['close'],'fiat': usdt,'coins': coin,'wallet': wallet,'drawBack':(wallet-lastAth)/lastAth}
        dt = dt.append(myrow,ignore_index=True)
  
  #Stop Loss
    elif row['low'] < stopLoss and coin > 0:

        sellPrice = stopLoss

        usdt = coin * sellPrice
        fee = makerFee * usdt
        usdt = usdt - fee
        coin = 0
        buyReady = False
        wallet = usdt
        if wallet > lastAth:
            lastAth = wallet
        # print("Sell COIN at Stop Loss",sellPrice,'$ the', index)
        myrow = {'date': index,'position': "Sell", 'reason': 'Sell Stop Loss', 'price': sellPrice, 'frais': fee, 'fiat': usdt, 'coins': coin, 'wallet': wallet, 'drawBack':(wallet-lastAth)/lastAth}
        dt = dt.append(myrow,ignore_index=True)

  #Take Profit
    elif row['high'] > takeProfit and coin > 0:

        sellPrice = takeProfit

        usdt = coin * sellPrice
        fee = makerFee * usdt
        usdt = usdt - fee
        coin = 0
        buyReady = False
        wallet = usdt
        if wallet > lastAth:
            lastAth = wallet
        # print("Sell COIN at Take Profit Loss",sellPrice,'$ the', index)
        myrow = {'date': index,'position': "Sell", 'reason': 'Sell Take Profit', 'price': sellPrice, 'frais': fee, 'fiat': usdt, 'coins': coin, 'wallet': wallet, 'drawBack':(wallet-lastAth)/lastAth}
        dt = dt.append(myrow,ignore_index=True)     

  # Sell Market
    elif sellCondition(row, previousRow):
        buyReady = True
    if coin > 0 and sellReady == True:
        sellPrice = row['close']
        usdt = coin * sellPrice
        frais = fee * usdt
        usdt = usdt - frais
        coin = 0
        wallet = usdt
        if wallet > lastAth:
            lastAth = wallet
        # print("Sell COIN at",sellPrice,'$ the', index)
        myrow = {'date': index,'position': "Sell", 'reason': 'Sell Market', 'price': sellPrice, 'frais': frais, 'fiat': usdt, 'coins': coin, 'wallet': wallet, 'drawBack':(wallet-lastAth)/lastAth}
        dt = dt.append(myrow,ignore_index=True)
    previousRow = row

#///////////////////////////////////////
print("Period : [" + str(dfTest.index[0]) + "] -> [" +str(dfTest.index[len(dfTest)-1]) + "]")
dt = dt.set_index(dt['date'])
dt.index = pd.to_datetime(dt.index)
dt['resultat'] = dt['wallet'].diff()
dt['resultat%'] = dt['wallet'].pct_change()*100
dt.loc[dt['position']=='Buy','resultat'] = None
dt.loc[dt['position']=='Buy','resultat%'] = None

dt['tradeIs'] = ''

dt.loc[dt['resultat']<=0,'tradeIs'] = 'Bad'
dt.loc[dt['resultat']>0,'tradeIs'] = 'Good'

iniClose = dfTest.iloc[0]['close']
lastClose = dfTest.iloc[len(dfTest)-1]['close']
holdPorcentage = ((lastClose - iniClose)/iniClose) * 100
algoPorcentage = ((wallet - initalWallet)/initalWallet) * 100
vsHoldPorcentage = ((algoPorcentage - holdPorcentage)/holdPorcentage) * 100

print("Starting balance : 1000 $")
print("Final balance :",round(wallet,2),"$")
print("Performance vs US Dollar :",round(algoPorcentage,2),"%")
print("Buy and Hold Performence :",round(holdPorcentage,2),"%")
print("Performance vs Buy and Hold :",round(vsHoldPorcentage,2),"%")
print("Number of negative trades : ",dt.groupby('tradeIs')['date'].nunique()['Bad'])
print("Number of positive trades : ",dt.groupby('tradeIs')['date'].nunique()['Good'])
print("Average Positive Trades : ",round(dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].sum()/dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].count(),2),"%")
print("Average Negative Trades : ",round(dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].sum()/dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].count(),2),"%")
idbest = dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].idxmax()
idworst = dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].idxmin()
print("Best trade +"+str(round(dt.loc[dt['tradeIs'] == 'Good', 'resultat%'].max(),2)),"%, the ",dt['date'][idbest])
print("Worst trade",round(dt.loc[dt['tradeIs'] == 'Bad', 'resultat%'].min(),2),"%, the ",dt['date'][idworst])
print("Worst drawBack", str(100*round(dt['drawBack'].min(),2)),"%")
print("Total fee : ",round(dt['frais'].sum(),2),"$")
reasons = dt['reason'].unique()
for r in reasons:
    print(r+" number :",dt.groupby('reason')['date'].nunique()[r])

dt[['wallet','price']].plot(subplots=True, figsize=(20,10))
print('PLOT')
dt

Period : [2020-01-30 04:00:00] -> [2021-10-25 11:00:00]
Starting balance : 1000 $
Final balance : 0.13 $
Performance vs US Dollar : -99.99 %
Buy and Hold Performence : 2270.06 %
Performance vs Buy and Hold : -104.4 %
Number of negative trades :  10402


KeyError: 'Good'